# Ejercicio 2 - Sumar matrices con kernel 2D

Partiendo del código suministrado:

* Completar los fragmentos indicados con `TODO` 



Sugerencias

1. Completar el kernel normalmente

2. Implementar dos versiones con acceso recorriendo filas y columnas y usar la variable booleana coalesced para poder elegir entre las dos

3. Comparar los tiempos de ejecución

4. El acceso es coalescente ¿por filas o por columnas?


---

In [ ]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [ ]:
import numpy as np
from numba import cuda

n = 4096

# TODO: completar
@cuda.jit
def matrix_add(a, b, out, coalesced):
    None

threads_per_block = (32, 32)  # 2D block
blocks = (128, 128) # 2D grid

h_a = np.arange(n*n).reshape(n,n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# TODO: copia a la GPU y reserva para la salida

# TODO: invocación kernel y obtención resultados
truth = h_a+h_b
np.testing.assert_equal(h_out, truth)

### Sergio

In [ ]:
import numpy as np
from numba import cuda

n = 4096

# TODO: completar
@cuda.jit
def matrix_add(a, b, out, coalesced):
    x, y = cuda.grid(2)
    if coalesced:
        out[y][x] = a[y][x] + b[y][x]
    else:
        out[x][y] = a[x][y] + b[x][y]
        

threads_per_block = (32, 32)  # 2D block
blocks = (128, 128) # 2D grid

h_a = np.arange(n*n).reshape(n,n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# TODO: copia a la GPU y reserva para la salida
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_out = cuda.device_array(shape=(n,n), dtype=np.float32)

# TODO: invocación kernel y obtención resultados
matrix_add[blocks, threads_per_block](d_a, d_b, d_out, False)
h_out = d_out.copy_to_host()
truth = h_a+h_b
np.testing.assert_equal(h_out, truth)

In [ ]:
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize()
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize()